In [ ]:
!pip install -q transformers4rec[nvtabular] protobuf==4.25.0 pyarrow==12.0.1 pandas-gbq==0.19.2 tritonclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 119.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.5/485.5 kB 40.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.5/284.5 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 73.4 MB/s eta 0:0

In [ ]:
!pip install -q cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.nvidia.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import gc
import calendar
import datetime

import cudf
import cupy
import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags

In [ ]:
from numba import config
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [ ]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/content/drive/MyDrive/Datasets/rsc15/raw")
FILENAME_PATTERN = 'rsc15-clicks.dat'
DATA_PATH = os.path.join(DATA_FOLDER, FILENAME_PATTERN)

OUTPUT_FOLDER = "./yoochoose_transformed"
!mkdir {OUTPUT_FOLDER}
OVERWRITE = False

USE_SYNTHETIC = os.environ.get("USE_SYNTHETIC", False)

In [ ]:
def generate_synthetic_data(
    start_date: datetime.date, end_date: datetime.date, rows_per_day: int = 10000
) -> pd.DataFrame:
    assert end_date > start_date, "end_date must be later than start_date"

    number_of_days = (end_date - start_date).days
    total_number_of_rows = number_of_days * rows_per_day

    # Generate a long-tail distribution of item interactions. This simulates that some items are
    # more popular than others.
    long_tailed_item_distribution = np.clip(
        np.random.lognormal(3.0, 1.0, total_number_of_rows).astype(np.int64), 1, 50000
    )

    # generate random item interaction features
    df = pd.DataFrame(
        {
            "session_id": np.random.randint(70000, 80000, total_number_of_rows),
            "item_id": long_tailed_item_distribution,
        },
    )

    # generate category mapping for each item-id
    df["category"] = pd.cut(df["item_id"], bins=334, labels=np.arange(1, 335)).astype(
        np.int64
    )

    max_session_length = 60 * 60  # 1 hour

    def add_timestamp_to_session(session: pd.DataFrame):
        random_start_date_and_time = calendar.timegm(
            (
                start_date
                # Add day offset from start_date
                + datetime.timedelta(days=np.random.randint(0, number_of_days))
                # Add time offset within the random day
                + datetime.timedelta(seconds=np.random.randint(0, 86_400))
            ).timetuple()
        )
        session["timestamp"] = random_start_date_and_time + np.clip(
            np.random.lognormal(3.0, 1.0, len(session)).astype(np.int64),
            0,
            max_session_length,
        )
        return session

    df = df.groupby("session_id").apply(add_timestamp_to_session).reset_index()

    return df

In [ ]:
if USE_SYNTHETIC:
    START_DATE = os.environ.get("START_DATE", "2014/4/1")
    END_DATE = os.environ.get("END_DATE", "2014/4/5")
    interactions_df = generate_synthetic_data(datetime.datetime.strptime(START_DATE, '%Y/%m/%d'),
                                              datetime.datetime.strptime(END_DATE, '%Y/%m/%d'))
    interactions_df = cudf.from_pandas(interactions_df)
else:
    interactions_df = cudf.read_csv(DATA_PATH, sep=',',
                                    names=['session_id','timestamp', 'item_id', 'category'],
                                    dtype=['int', 'datetime64[s]', 'int', 'int'])

In [ ]:
print("Count with in-session repeated interactions: {}".format(len(interactions_df)))

# Sorts the dataframe by session and timestamp, to remove consecutive repetitions
interactions_df.timestamp = interactions_df.timestamp.astype(int)
interactions_df = interactions_df.sort_values(['session_id', 'timestamp'])
past_ids = interactions_df['item_id'].shift(1).fillna()
session_past_ids = interactions_df['session_id'].shift(1).fillna()

# Keeping only no consecutive repeated in session interactions
interactions_df = interactions_df[~((interactions_df['session_id'] == session_past_ids) & (interactions_df['item_id'] == past_ids))]

print("Count after removed in-session repeated interactions: {}".format(len(interactions_df)))

Count with in-session repeated interactions: 33003944
Count after removed in-session repeated interactions: 28971543


In [ ]:
items_first_ts_df = interactions_df.groupby('item_id').agg({'timestamp': 'min'}).reset_index().rename(columns={'timestamp': 'itemid_ts_first'})
interactions_merged_df = interactions_df.merge(items_first_ts_df, on=['item_id'], how='left')
print(interactions_merged_df.head())

   session_id   timestamp    item_id  category  itemid_ts_first
0        4993  1396727816  214835285         0       1396332436
1        4993  1396727863  214530703         0       1396339114
2        4993  1396727898  214530705         0       1396330224
3        4993  1396728063  214835713         0       1396327474
4        4993  1396730097  214512611         0       1396328044


In [ ]:
if os.path.isdir(DATA_FOLDER) == False:
    os.mkdir(DATA_FOLDER)
interactions_merged_df.to_parquet(os.path.join(DATA_FOLDER, 'interactions_merged_df.parquet'))

In [ ]:

# print the total number of unique items in the dataset
print(interactions_merged_df.item_id.nunique())

52739


In [ ]:
# free gpu memory
del interactions_df, session_past_ids, items_first_ts_df
gc.collect()

0

In [ ]:
# Encodes categorical features as contiguous integers
cat_feats = ColumnSelector(['session_id', 'category', 'item_id']) >> nvt.ops.Categorify()

# create time features
session_ts = ColumnSelector(['timestamp'])
session_time = (
    session_ts >>
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >>
    nvt.ops.Rename(name = 'event_time_dt')
)
sessiontime_weekday = (
    session_time >>
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >>
    nvt.ops.Rename(name ='et_dayofweek')
)

# Derive cyclical features: Define a custom lambda function
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

weekday_sin = sessiontime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_sin')

# Compute Item recency: Define a custom Op
class ItemRecency(nvt.ops.Operator):
    def transform(self, columns, gdf):
        for column in columns.names:
            col = gdf[column]
            item_first_timestamp = gdf['itemid_ts_first']
            delta_days = (col - item_first_timestamp) / (60*60*24)
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf

    def compute_selector(
        self,
        input_schema: Schema,
        selector: ColumnSelector,
        parents_selector: ColumnSelector,
        dependencies_selector: ColumnSelector,
    ) -> ColumnSelector:
        self._validate_matching_cols(input_schema, parents_selector, "computing input selector")
        return parents_selector

    def column_mapping(self, col_selector):
        column_mapping = {}
        for col_name in col_selector.names:
            column_mapping[col_name + "_age_days"] = [col_name]
        return column_mapping

    @property
    def dependencies(self):
        return ["itemid_ts_first"]

    @property
    def output_dtype(self):
        return np.float64

recency_features = session_ts >> ItemRecency()
# Apply standardization to this continuous feature
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize(out_dtype=np.float32) >> nvt.ops.Rename(name='product_recency_days_log_norm')

time_features = (
    session_time +
    sessiontime_weekday +
    weekday_sin +
    recency_features_norm
)

features = ColumnSelector(['session_id', 'timestamp']) + cat_feats + time_features

In [ ]:
# Define Groupby Operator
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["session_id"],
    sort_cols=["timestamp"],
    aggs={
        'item_id': ["list", "count"],
        'category': ["list"],
        'timestamp': ["first"],
        'event_time_dt': ["first"],
        'et_dayofweek_sin': ["list"],
        'product_recency_days_log_norm': ["list"]
        },
    name_sep="-")

# Truncate sequence features to first interacted 20 items
SESSIONS_MAX_LENGTH = 20

item_feat = groupby_features['item_id-list'] >> nvt.ops.TagAsItemID()
cont_feats = groupby_features['et_dayofweek_sin-list', 'product_recency_days_log_norm-list'] >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])


groupby_features_list =  item_feat + cont_feats + groupby_features['category-list']
groupby_features_truncated = groupby_features_list >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH)

# Calculate session day index based on 'event_time_dt-first' column
day_index = ((groupby_features['event_time_dt-first'])  >>
             nvt.ops.LambdaOp(lambda col: (col - col.min()).dt.days +1) >>
             nvt.ops.Rename(f = lambda col: "day_index") >>
             nvt.ops.AddMetadata(tags=[Tags.CATEGORICAL])
            )

# tag session_id column for serving with legacy api
sess_id = groupby_features['session_id'] >> nvt.ops.AddMetadata(tags=[Tags.CATEGORICAL])

# Select features for training
selected_features = sess_id + groupby_features['item_id-count'] + groupby_features_truncated + day_index

# Filter out sessions with less than 2 interactions
MINIMUM_SESSION_LENGTH = 2
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

In [ ]:
from numba import config
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [ ]:
dataset = nvt.Dataset(interactions_merged_df)
workflow = nvt.Workflow(filtered_sessions)
# Learns features statistics necessary of the preprocessing workflow
workflow.fit(dataset)
# Apply the preprocessing workflow in the dataset and converts the resulting Dask cudf dataframe to a cudf dataframe
sessions_gdf = workflow.transform(dataset).compute()

In [ ]:
sessions_gdf.head()

,session_id,item_id-count,item_id-list,et_dayofweek_sin-list,product_recency_days_log_norm-list,category-list,day_index
0,1,3,"[7342, 19525, 15501]","[0.7818320421108522, 0.7818320421108522, 0.781...","[-1.2986504, -1.3283174, -1.354854]","[3, 3, 3]",7
1,2,5,"[1970, 8041, 5180, 9412, 607]","[0.7818320421108522, 0.7818320421108522, 0.781...","[-1.3023797, -2.2679935, -2.17633, -2.1693697,...","[3, 3, 3, 3, 3]",7
2,3,3,"[83, 2163, 2344]","[0.43388293040961884, 0.43388293040961884, 0.4...","[-2.2485673, -2.3008614, -2.2673779]","[3, 3, 3]",2
3,4,2,"[2465, 5662]","[0.7818320421108522, 0.7818320421108522]","[-1.283754, -1.2788646]","[3, 3]",7
4,6,2,"[3214, 208]","[8.975979006501142e-07, 8.975979006501142e-07]","[-1.3856947, -1.3733647]","[3, 3]",6


In [ ]:
workflow.save('workflow_etl')

In [ ]:
sessions_gdf = sessions_gdf[sessions_gdf.day_index>=178]

In [ ]:
from merlin_standard_lib import Schema
SCHEMA_PATH = "/content/drive/MyDrive/Datasets/rsc15/raw/processed_nvt/schema.pbtxt"
schema = Schema().from_proto_text(SCHEMA_PATH)

In [ ]:
dataset = nvt.Dataset(interactions_merged_df)
workflow = nvt.Workflow(filtered_sessions)
# Learn features statistics necessary of the preprocessing workflow
# The following will generate schema.pbtxt file in the provided folder and export the parquet files.
workflow.fit_transform(dataset).to_parquet(os.path.join(DATA_FOLDER, "processed_nvt"))

In [ ]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,workflow_etl/categories/unique.session_id.parquet,0.0,9249731.0,session_id,9249732.0,512.0,NaN,NaN
1,item_id-count,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,workflow_etl/categories/unique.item_id.parquet,0.0,52741.0,item_id,52742.0,512.0,NaN,NaN
2,item_id-list,"(Tags.CATEGORICAL, Tags.LIST, Tags.ID, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,workflow_etl/categories/unique.item_id.parquet,0.0,52741.0,item_id,52742.0,512.0,0.0,20.0
3,et_dayofweek_sin-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0
4,product_recency_days_log_norm-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0
5,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,workflow_etl/categories/unique.category.parquet,0.0,336.0,category,337.0,42.0,0.0,20.0
6,day_index,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
workflow.save(os.path.join(DATA_FOLDER, "workflow_etl"))

In [ ]:
# read in the processed train dataset
sessions_gdf = cudf.read_parquet(os.path.join(DATA_FOLDER, "processed_nvt/part_0.parquet"))
if USE_SYNTHETIC:
    THRESHOLD_DAY_INDEX = int(os.environ.get("THRESHOLD_DAY_INDEX", '1'))
    sessions_gdf = sessions_gdf[sessions_gdf.day_index>=THRESHOLD_DAY_INDEX]
else:
    sessions_gdf = sessions_gdf[sessions_gdf.day_index>=178]

In [ ]:
print(sessions_gdf.head(3))

         session_id  item_id-count  \
6606147    11255549             12   
6606148    11255552              2   
6606149    11255553              2   

                                              item_id-list  \
6606147  [605, 879, 743, 91, 4778, 1584, 3447, 8084, 34...   
6606148                                       [185, 12289]   
6606149                                       [7300, 1954]   

                                     et_dayofweek_sin-list  \
6606147  [-0.43388454782514785, -0.43388454782514785, -...   
6606148       [-0.43388454782514785, -0.43388454782514785]   
6606149         [-0.7818309228245777, -0.7818309228245777]   

                        product_recency_days_log_norm-list  \
6606147  [1.5241561, 1.523876, 1.523935, 1.5241641, 1.5...   
6606148                              [-0.533007, 1.521495]   
6606149                             [1.5338274, 1.5355083]   

                                category-list  day_index  
6606147  [4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4

In [ ]:
from transformers4rec.utils.data_utils import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir=os.path.join(DATA_FOLDER, "preproc_sessions_by_day"),
                       partition_col='day_index',
                       timestamp_col='session_id',
                      )

Creating time-based splits: 100%|██████████| 5/5 [00:02<00:00,  2.19it/s]


In [ ]:
# free gpu memory
del  sessions_gdf
gc.collect()

845

end2end

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/content/drive/MyDrive/Datasets/rsc15/raw")
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/preproc_sessions_by_day")

In [ ]:
from merlin.schema import Schema
from merlin.io import Dataset

train = Dataset(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))
schema = train.schema

In [ ]:
schema = schema.select_by_name(
   ['item_id-list', 'category-list', 'product_recency_days_log_norm-list', 'et_dayofweek_sin-list']
)

In [ ]:
schema

,name,tags,dtype,is_list,is_ragged,properties.freq_threshold,properties.num_buckets,properties.cat_path,properties.max_size,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,item_id-list,"(Tags.CATEGORICAL, Tags.LIST, Tags.ID, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,0.0,NaN,workflow_etl/categories/unique.item_id.parquet,0.0,512.0,52742.0,0.0,52741.0,item_id,0,20
1,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,0.0,NaN,workflow_etl/categories/unique.category.parquet,0.0,42.0,337.0,0.0,336.0,category,0,20
2,product_recency_days_log_norm-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20
3,et_dayofweek_sin-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float64', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20


In [ ]:
from transformers4rec import torch as tr

max_sequence_length, d_model = 20, 320
# Define input module to process tabular input-features and to prepare masked inputs
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=max_sequence_length,
    continuous_projection=64,
    aggregation="concat",
    d_output=d_model,
    masking="mlm",
)

# Define Next item prediction-task
prediction_task = tr.NextItemPredictionTask(weight_tying=True)

# Define the config of the XLNet Transformer architecture
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=8, n_layer=2, total_seq_length=max_sequence_length
)

# Get the end-to-end model
model = transformer_config.to_torch_model(input_module, prediction_task)

In [ ]:
BATCH_SIZE_TRAIN = int(os.environ.get("BATCH_SIZE_TRAIN", "512"))
BATCH_SIZE_VALID = int(os.environ.get("BATCH_SIZE_VALID", "256"))
training_args = tr.trainer.T4RecTrainingArguments(
            dataloader_drop_last = True,
            gradient_accumulation_steps = 1,
            output_dir="./tmp",
            max_sequence_length=20,
            data_loader_engine='merlin',
            num_train_epochs=10,
            lr_scheduler_type='cosine',
            # dataloader_drop_last=False,
            per_device_train_batch_size = BATCH_SIZE_TRAIN,
            per_device_eval_batch_size = BATCH_SIZE_VALID,
            learning_rate=0.0005,
            fp16=True,
            report_to = [],
            logging_steps=200
        )

In [ ]:
trainer = tr.Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True)

In [ ]:
from transformers4rec.torch.utils.examples_utils import fit_and_evaluate
start_time_idx = int(os.environ.get("START_TIME_INDEX", "179"))
end_time_idx = int(os.environ.get("END_TIME_INDEX", "182"))
OT_results = fit_and_evaluate(recsys_trainer, start_time_index=start_time_idx, end_time_index=end_time_idx, input_dir=OUTPUT_DIR)


***** Launch training for day 179: *****


Step,Training Loss
200,6.953600


AttributeError: ignored

multigpu

In [ ]:
TRAINER_FILE = os.path.join(os.environ.get("INPUT_DATA_DIR", "/content/drive/MyDrive/Datasets/rsc15/raw"), "pyt_trainer.py")

In [ ]:
%%writefile {TRAINER_FILE}

import argparse
import os
import glob
import torch

import cupy

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
from merlin.schema import Schema
from merlin.io import Dataset

cupy.cuda.Device(int(os.environ["LOCAL_RANK"])).use()

# define arguments that can be passed to this python script
parser = argparse.ArgumentParser(description='Hyperparameters for model training')
parser.add_argument('--path', type=str, help='Directory with training and validation data')
parser.add_argument('--learning-rate', type=float, default=0.0005, help='Learning rate for training')
parser.add_argument('--per-device-train-batch-size', type=int, default=64, help='Per device batch size for training')
parser.add_argument('--per-device-eval-batch-size', type=int, default=32, help='Per device batch size for evaluation')
sh_args = parser.parse_args()

# create the schema object by reading the processed train set generated in the previous 01-ETL-with-NVTabular notebook

INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data")
train = Dataset(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))
schema = train.schema

# select the subset of features we want to use for training the model by their tags or their names.
schema = schema.select_by_name(
   ['item_id-list', 'category-list', 'product_recency_days_log_norm-list', 'et_dayofweek_sin-list']
)

max_sequence_length, d_model = 20, 320
# Define input module to process tabular input-features and to prepare masked inputs
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=max_sequence_length,
    continuous_projection=64,
    aggregation="concat",
    d_output=d_model,
    masking="mlm",
)

# Define Next item prediction-task
prediction_task = tr.NextItemPredictionTask(weight_tying=True)

# Define the config of the XLNet Transformer architecture
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=8, n_layer=2, total_seq_length=max_sequence_length
)

# Get the end-to-end model
model = transformer_config.to_torch_model(input_module, prediction_task)

# Set training arguments
training_args = tr.trainer.T4RecTrainingArguments(
            output_dir="./tmp",
            max_sequence_length=20,
            data_loader_engine='merlin',
            num_train_epochs=10,
            dataloader_drop_last=True,
            per_device_train_batch_size = sh_args.per_device_train_batch_size,
            per_device_eval_batch_size = sh_args.per_device_eval_batch_size,
            learning_rate=sh_args.learning_rate,
            report_to = [],
            logging_steps=200,
        )

# Instantiate the trainer
recsys_trainer = tr.Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True)

# Set input and output directories
INPUT_DIR=sh_args.path
OUTPUT_DIR=sh_args.path

import time
start = time.time()

# main loop for training
start_time_window_index = int(os.environ.get("START_TIME_INDEX", "178"))
final_time_window_index = int(os.environ.get("END_TIME_INDEX", "181"))

# Iterating over days from 178 to 181
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))

    # Train on day related to time_index
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    recsys_trainer.train_dataset_or_path = train_paths
    recsys_trainer.reset_lr_scheduler()
    recsys_trainer.train()
    recsys_trainer.state.global_step +=1
    print('finished')

    # Evaluate on the following day
    recsys_trainer.eval_dataset_or_path = eval_paths
    eval_metrics = recsys_trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(eval_metrics.keys()):
        print(" %s = %s" % (key, str(eval_metrics[key])))
    wipe_memory()

# export evaluation metrics to a file
import json
fname = os.path.join(INPUT_DATA_DIR, "eval_metrics.txt")
f = open(fname, "w")
f.write(json.dumps(eval_metrics))
f.close()

end = time.time()
print('Total training time:', end-start)

Overwriting /content/drive/MyDrive/Datasets/rsc15/raw/pyt_trainer.py


In [ ]:
from torch.cuda import device_count
num_gpus = device_count()
NUM_PROCESSES = min(num_gpus, 2)

In [ ]:
import os
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{DATA_FOLDER}/preproc_sessions_by_day")
LR = float(os.environ.get("LEARNING_RATE", "0.0005"))
BATCH_SIZE_TRAIN = int(os.environ.get("BATCH_SIZE_TRAIN", "256"))
BATCH_SIZE_VALID = int(os.environ.get("BATCH_SIZE_VALID", "128"))

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!python -m torch.distributed.run --nproc_per_node {NUM_PROCESSES} {TRAINER_FILE} --path {OUTPUT_DIR} --learning-rate {LR} --per-device-train-batch

2023-11-12 18:52:06.121409: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 18:52:06.121469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 18:52:06.121508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 18:52:11.708901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: pyt_trainer.py [-h] [--path PATH] [--learning-rate LEARNING_RATE]
                      [--per-device-train-batch-size PER_DEVICE_TRAIN_BATCH_SIZE]
                      [--per-device-eval-batch-size PER_DEVICE_EVAL_BATCH_SIZE]
pyt